# Generate Monthly fCO2 (fugacity) from Monthly CO2STAR and CO2_alpha

In [1]:
import glob
import os.path
import pprint

import distributed
import ncar_jobqueue
import numpy as np
import xarray as xr
import yaml

from utils import path_replace
from utils_units import conv_units

/glade/work/klindsay/miniconda3/envs/GCB_2022/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
xr.set_options(keep_attrs=True);

In [3]:
with open("GCB_metadata.yaml", mode="r") as fptr:
    GCB_metadata = yaml.safe_load(fptr)
pprint.pprint(GCB_metadata)

{'A': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001',
                 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.002']},
 'B': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRC.001']},
 'C': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRC.001']},
 'D': {'cases': ['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRD.001',
                 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BCRD.002']}}


In [4]:
tseries_root = "/glade/campaign/cesm/development/bgcwg/projects/GCB_2022"

In [5]:
def gen_fCO2_driver(GCB_name, CESM_cases):
    print(GCB_name)
    print(CESM_cases)

    for case in CESM_cases:
        dir = f"{tseries_root}/{case}/output/ocn/proc/tseries/month_1"
        co2star_paths = glob.glob(f"{dir}/{case}.pop.h.CO2STAR.*.nc")
        co2star_paths.sort()
        co2_alpha_paths = glob.glob(f"{dir}/{case}.pop.h.CO2_alpha.*.nc")
        co2_alpha_paths.sort()
        for (co2star_path, co2_alpha_path) in zip(co2star_paths, co2_alpha_paths):
            ds = xr.open_dataset(co2star_path, chunks={"time": 12})
            ds["CO2_alpha"] = xr.open_dataset(
                co2_alpha_path, chunks={"time": 12}
            )["CO2_alpha"]

            ds_out = ds.drop_vars(["CO2STAR", "CO2_alpha"])
            ds_out["fCO2"] = gen_fCO2(ds)

            # ensure NaN _FillValues do not get generated
            for d in [ds_out.variables, ds_out.coords]:
                for var in d:
                    if "_FillValue" not in ds_out[var].encoding:
                        ds_out[var].encoding["_FillValue"] = None

            fCO2_path = path_replace(
                co2star_path, base_replace={"CO2STAR": "fCO2"}
            )

            print(f"writing {fCO2_path}")
            ds_out.to_netcdf(fCO2_path)


def gen_fCO2(ds):
    """compute fCO2 (fugacity) from CO2STAR and CO2_alpha (solubility)"""

    # compute fCO2 and set metadata
    fCO2 = ds["CO2STAR"] / ds["CO2_alpha"]
    fCO2.attrs["long_name"] = "Surface CO2 Fugacity"
    units_numer = ds["CO2STAR"].attrs["units"]
    units_denom = ds["CO2_alpha"].attrs["units"]
    fCO2.attrs["units"] = f"({units_numer})/({units_denom})"
    fCO2.encoding = ds["CO2STAR"].encoding

    # divide by reference density
    rho_sw_val = ds["rho_sw"].values[()]
    rho_sw_units = ds["rho_sw"].units
    fCO2.attrs["units"] += f"/({rho_sw_val} {rho_sw_units})"

    # convert to requested units
    fCO2 = conv_units(fCO2, "uatm")

    return fCO2


# Obtain Computational Resources

In [6]:
cluster = ncar_jobqueue.NCARCluster(
    cores=1,  # The number of cores you want
    memory='4GB',  # Amount of memory
    processes=1,  # How many processes
    walltime='01:00:00',  # Amount of wall time
)

cluster.scale(8)

client = distributed.Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.48:38237,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/klindsay/GCB_2022/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
for GCB_name in GCB_metadata:
    gen_fCO2_driver(GCB_name, GCB_metadata[GCB_name]["cases"])

A
['g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001', 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.002']
writing /glade/campaign/cesm/development/bgcwg/projects/GCB_2022/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001/output/ocn/proc/tseries/month_1/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.fCO2.177801-180212.nc
writing /glade/campaign/cesm/development/bgcwg/projects/GCB_2022/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001/output/ocn/proc/tseries/month_1/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.fCO2.180301-182712.nc
writing /glade/campaign/cesm/development/bgcwg/projects/GCB_2022/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001/output/ocn/proc/tseries/month_1/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001.pop.h.fCO2.182801-185212.nc
writing /glade/campaign/cesm/development/bgcwg/projects/GCB_2022/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.GCB_2022.BDRD.001/output/ocn/proc/tser

# Release Computational Resources

In [8]:
client.close()
cluster.close()
